# 事前の設定（GPUの設定）
* 上のメニュー「ランタイム」 -> ランタイムのタイプを変更」を選択する
* ハードウェアアクセラレータを「GPU」に変更し，保存する

In [1]:
!nvidia-smi    # GPUの確認, '!'を先頭につけるとシェルコマンドになる

Mon Jan  7 01:11:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    69W / 149W |    626MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Pytorch（Deep Learningのフレームワーク）のインストール
* http://pytorch.org/

In [2]:
!pip install torch torchvision

# PILの再インストール

In [3]:
!pip install pillow==4.1.1

# GPUが使えているかの確認

In [0]:
import torch
torch.cuda.is_available()

# ImageNet 学習済みモデルによる画像認識
* https://pytorch.org/docs/stable/torchvision/models.html

In [4]:
import torchvision.models as models
cnn_model = models.vgg19(pretrained=True)    # 学習済みモデルの読み込み
cnn_model.eval()    # 評価モードにする

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:16<00:00, 34007258.60it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

# ラベル名の取得

In [5]:
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

--2019-01-07 01:12:01--  https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.234.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.234.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [application/octet-stream]
Saving to: ‘imagenet_class_index.json’

imagenet_class_inde 100%[===================>]  34.53K  --.-KB/s    in 0.03s   

2019-01-07 01:12:02 (1.08 MB/s) - ‘imagenet_class_index.json’ saved [35363/35363]



In [0]:
import json
class_index = json.load(open('imagenet_class_index.json', 'r'))
labels = {int(key):value for (key, value) in class_index.items()}

# 予測のための関数の定義

In [0]:
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def predict(cnn_model, labels, image_file):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                     std=[0.229, 0.224, 0.225])
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224), transforms.ToTensor(), normalize])

    print('Read ' + image_file)
    img = Image.open(image_file, 'r')
    
    # exif情報を使った画像回転の補正
    if hasattr(img, '_getexif'):
        _orientation = img._getexif()[274]
        if _orientation == 6:
            img = img.rotate(-90, expand=True)
        elif _orientation == 8:
            img = img.rotate(-270, expand=True)
        elif _orientation == 3:
            img = img.rotate(180, expand=True)
    
    # 画像の前処理
    img = img.convert('RGB')
    img_tensor = preprocess(img)
    img_tensor.unsqueeze_(0)

    # 畳み込みニューラルネットワークの実行
    out = cnn_model(Variable(img_tensor))
    
    out = nn.functional.softmax(out, dim=1)    # 出力を確率にする
    out = out.data[0].numpy()

    rank_id = np.argsort(out)[::-1][:5]

    for i in range(5):
        print('%d: '%(i) + labels[rank_id[i]][1] + ' (' + str(out[rank_id[i]]) + ')')
    
    plt.imshow(np.array(img))
    plt.grid(False)
    plt.tick_params(labelbottom="off",bottom="off")    # x軸の削除
    plt.tick_params(labelleft="off",left="off")    # y軸の削除
    #print(np.array(img).shape)
    
    return

# 画像ファイルのアップロード
* 「ファイルの選択」から画像ファイルをアップロードする

In [0]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 画像認識の実行

In [0]:
predict(cnn_model, labels, file_name)